## Agreements

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [2]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
     users.append(agg['user__username'])   
users.remove("jmperez")
users

['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

In [31]:
import pandas as pd
from api.models import BatchAssignment


#df = pd.DataFrame({"name":users})
#df.set_index("name", inplace=True)
df = pd.DataFrame(columns=users)

for batch_assignment in BatchAssignment.objects.all():
    user_name = batch_assignment.user.username
    if user_name in users:
        batch_name = batch_assignment.batch.name
        value = "completed" if batch_assignment.done else "progressing"
        df.loc[batch_name, user_name] = value
df.fillna("na", inplace=True)

df.reset_index(inplace=True)
df = df.rename({"index": "batch_name"}, axis=1)

df.to_dict('records'), users

([{'batch_name': 'training',
   'asilva': 'progressing',
   'dvalado': 'completed',
   'fdesanctis': 'progressing',
   'gclerici': 'progressing',
   'gdamill': 'completed',
   'lprats': 'progressing'}],
 ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats'])